In [15]:
import cv2
import os
from PIL import Image

In [11]:
image_dir = "Dataset/test/images"
label_dir = "Dataset/test/labels"
output_dir = "Dataset/out_test"

In [12]:
def extract_crops(image_dir, label_dir, output_dir):
    """
    Extracts crops from images based on YOLOv7 labels and organizes them by class.

    Args:
        image_dir: Directory containing images.
        label_dir: Directory containing corresponding YOLOv7 label files (.txt).
        output_dir: Directory to save extracted crops (subdirectories will be created for each class).
    """

    os.makedirs(output_dir, exist_ok=True)

    for image_file in os.listdir(image_dir):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        image_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, os.path.splitext(image_file)[0] + '.txt')

        if not os.path.exists(label_path):
            print(f"Warning: Label file not found for {image_file}")
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Could not read image {image_file}")
            continue

        height, width, _ = image.shape

        with open(label_path, 'r') as f:
            for line in f:
                try:
                    class_id, x_center, y_center, w, h = map(float, line.split())
                    class_id = int(class_id)  # Ensure class_id is an integer

                    # Create class-specific output directory
                    class_output_dir = os.path.join(output_dir, str(class_id)) # Directory name is class_id
                    os.makedirs(class_output_dir, exist_ok=True)

                    x_center *= width
                    y_center *= height
                    w *= width
                    h *= height

                    x1 = int(max(0, x_center - w / 2))
                    y1 = int(max(0, y_center - h / 2))
                    x2 = int(min(width, x_center + w / 2))
                    y2 = int(min(height, y_center + h / 2))

                    crop = image[y1:y2, x1:x2]

                    if crop.size > 0:
                        crop_filename = f"{os.path.splitext(image_file)[0]}_{class_id}.jpg"
                        crop_path = os.path.join(class_output_dir, crop_filename) # Save in class directory
                        cv2.imwrite(crop_path, crop)
                    else:
                         print(f"Warning: Empty crop for {image_file}, class {class_id}. Skipping.")

                except ValueError:
                    print(f"Error: Invalid line format in label file for {image_file}: {line.strip()}")


In [13]:
extract_crops(image_dir, label_dir, output_dir)

In [24]:
output_dir = "Dataset/out_valid"

In [25]:
def resize_images(data_dir, new_size):
    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Add other extensions if needed
                filepath = os.path.join(root, file)
                try:
                    img = Image.open(filepath).convert("RGB")  # Open and convert to RGB
                    img = img.resize(new_size, Image.LANCZOS) # Resize with antialiasing
                    img.save(filepath) # Save the resized image (overwrite original)
                except Exception as e:
                    print(f"Error processing {filepath}: {e}")

image_size = (32, 32) # Example size
resize_images(output_dir, image_size)

In [26]:
img = Image.open('Dataset/out_valid/10/12-2-1b_PNG.rf.91afaa894bbbd91595911506b9ec46e2_10.jpg')
print(img.size)

(32, 32)
